# Ex 0. Getting Started: standard Hopfield network

**0.1.** Write a method that generates binary balanced random patterns; and a method that computes the next state S(t + 1) of the network, given the current state S(t) = (S1(t), . . . , SN (t)) and a set of patterns P1,...,PM, according to eqs.(1)-(2).

In [1]:
import numpy as np

In [8]:
def generate_patterns(M, N):
    """
    Args:
        M: number of patterns
        N: number of neurons
    Output:
        Generates binary balanced random patterns.
    """
    return np.random.choice([-1, 1], size=(M, N))
#initial_sequences = [np.random.choice([-1, 1], size=N) for _ in range(M)]
#seqs = np.stack(initial_sequences, axis=0)

def next_state(S, seqs, beta=4):
    """
    Args:
        S: state of the sequence
        seqs: the sequences of the patterns
        beta: constant for the tanh function
    Output:
        Updates the sequence and outputs the new sequence and the weights
        of the interactions between spins in the sequence.
    """
    W = np.zeros((len(S), len(S)))
    for i in range(len(S)):
        for j in range(len(S)):
            W[i, j] = 1 / len(S) * np.sum((seqs[:, i]) * (seqs[:, j]))
    new_S = np.tanh(beta*(W@S))
    return new_S, W
    

In [9]:
N = 100  # Number of neurons
M = 5  # Number of patterns
patterns = generate_patterns(M, N)

# flip 5% of indices
initial_state = patterns[0,:].copy()
c = 0.05
flip_indices = np.random.choice(len(initial_state), int(N*c), replace=False)
initial_state[list(flip_indices)] *= -1

print("First pattern sum:",np.sum(patterns[0,:]), patterns[0,:])
print("Intial state sum:", np.sum(initial_state), initial_state)

First pattern sum: -6 [ 1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1  1 -1  1  1  1 -1 -1 -1  1 -1  1 -1
 -1 -1  1 -1 -1 -1  1 -1  1 -1  1 -1  1 -1  1  1 -1 -1 -1  1 -1 -1  1 -1
 -1 -1 -1  1 -1  1  1 -1  1  1 -1  1  1  1 -1 -1 -1 -1 -1  1  1 -1  1  1
 -1  1  1  1 -1  1  1  1 -1  1  1  1 -1 -1 -1  1 -1 -1  1  1  1  1 -1  1
  1 -1 -1  1]
Intial state sum: -4 [ 1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1  1 -1  1  1  1 -1 -1 -1  1 -1  1 -1
  1 -1  1  1 -1 -1  1 -1  1 -1  1 -1  1 -1  1  1 -1 -1 -1  1 -1 -1  1 -1
 -1 -1 -1  1 -1  1 -1 -1  1  1 -1  1  1  1 -1 -1 -1 -1 -1  1  1 -1  1  1
 -1  1  1  1 -1  1  1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1  1  1  1 -1  1
  1 -1 -1  1]


In [10]:
# evolution of the network after n steps
n = 15
S = initial_state
for i in range(n):
    S, __ = next_state(S, patterns, beta=4)
    print("Similarity between the 1st pattern and the current state:", round(np.dot(S,patterns[0,:])/N,4))

Similarity between the 1st pattern and the current state: 0.9966
Similarity between the 1st pattern and the current state: 0.999
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991
Similarity between the 1st pattern and the current state: 0.9991


In [ ]:
for m in range(M):
    print(f"Similarity between {m+1} pattern and final state: {round(np.abs(np.dot(S,patterns[m,:])/N),4)}")

Similarity between 1 pattern and final state: 0.9986
Similarity between 2 pattern and final state: 0.0607
Similarity between 3 pattern and final state: 0.0
Similarity between 4 pattern and final state: 0.121
Similarity between 5 pattern and final state: 0.0809


## Alternative implementation

In [2]:
import numpy as np

class HopfieldNetwork:
    def __init__(self, N):
        self.N = N
        self.weights = np.zeros((N, N))
    
    def generate_patterns(self, M):
        """Generates M random patterns with components +1 or -1 with equal probability."""
        return np.random.choice([-1, 1], (M, self.N))
    
    def train(self, patterns):
        """Trains the network using Hebbian learning rule."""
        M = patterns.shape[0]
        for i in range(self.N):
            for j in range(self.N):
                if i != j:
                    self.weights[i, j] = np.sum(patterns[:, i] * patterns[:, j]) / self.N
                else:
                    self.weights[i, j] = 0
    
    def update_state(self, state, beta=4):
        """Updates the network state using the hyperbolic tangent function."""
        h = np.dot(self.weights, state)
        return np.tanh(beta * h)
    
    def run_dynamics(self, initial_state, steps=10):
        """Simulates the network dynamics for a number of steps."""
        state = initial_state.copy()
        for _ in range(steps):
            state = self.update_state(state)
        return state
    
    def calculate_overlap(self, state, pattern):
        """Calculates the overlap between the current state and a given pattern."""
        return np.dot(state, pattern) / self.N

# Parameters
N = 100  # Number of neurons
M = 5    # Number of patterns
c = 0.05 # Percentage of neurons to flip

# Initialize the Hopfield network
network = HopfieldNetwork(N)

# Generate patterns and train the network
patterns = network.generate_patterns(M)
network.train(patterns)

# Set initial state close to the first pattern
initial_state = patterns[0].copy()
flip_indices = np.random.choice(N, int(N * c), replace=False)
initial_state[flip_indices] *= -1  # Flip the sign of selected neurons

# Run the dynamics of the network
final_state = network.run_dynamics(initial_state, steps=20)

# Evaluate the result
overlaps = [network.calculate_overlap(final_state, p) for p in patterns]
print("Overlaps with each pattern:", overlaps)


Overlaps with each pattern: [0.9985530189855538, -0.0403246446118404, -0.00015715294139141635, -0.08080838344970578, -0.08076846576299174]
